In [ ]:
# 回到 FINRL/
%cd ..

In [ ]:
from AI_Trading.src import config
from AI_Trading.src.env_portfolio_allocation import *
from AI_Trading.src.evaluate import *
from AI_Trading.src import model_config
from AI_Trading.src.preprocess import *
from AI_Trading.src.testPortfolio import *
from AI_Trading.src.train import *
from AI_Trading.src.augmentation import *
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3
import pandas as pd
from tqdm import tqdm
from warnings import simplefilter
simplefilter(action='ignore')
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

In [ ]:
# train
for i in tqdm(range(13)):
    train,trade = preprocess(config.TRAIN_START_DATE[i], config.TRAIN_END_DATE[i], config.TEST_START_DATE[i], config.TEST_END_DATE[i])
    env_kwargs = {
        "hmax": 100, 
        "initial_amount": config.INITIAL_AMOUNT, 
        "transaction_cost_pct": 0.001, 
        "state_space": len(train.tic.unique()), 
        "stock_dim": len(train.tic.unique()), 
        "tech_indicator_list": config.INDICATORS, 
        "action_space": len(train.tic.unique()), 
        "reward_scaling": 1e-4,
    }
    # env_train = portfolioAllocationEnv(df = train, **env_kwargs)
    # env_trade = portfolioAllocationEnv(df = trade, **env_kwargs)

    env_train = mddConcernEnv(df = train, **env_kwargs)
    env_trade = mddConcernEnv(df = trade, **env_kwargs)

    # train
    model_index = i
    model_name = 'A2C'
    exp = 'mdd'
    trainPortfolioAllocation(exp, env_train, model_name, model_index)

In [ ]:
model_name = 'A2C'
exp = 'mdd'

In [ ]:
training_log_path = f'{config.LOG_PATH}training_log_{exp}.csv'
if os.path.exists(training_log_path):
     os.remove(training_log_path)

df_rank_allYear = pd.DataFrame()
df_annualReturn_allYear = pd.DataFrame()
df_mdd_allYear = pd.DataFrame()
df_sharpe_allYear = pd.DataFrame()
df_sortino_allYear = pd.DataFrame()

# evaluate
for i in tqdm(range(3)):
    train,trade = preprocess(config.TRAIN_START_DATE[i], config.TRAIN_END_DATE[i], config.TEST_START_DATE[i], config.TEST_END_DATE[i])
    env_kwargs = {
        "training_log_path": training_log_path,
        "hmax": 100, 
        "initial_amount": config.INITIAL_AMOUNT, 
        "transaction_cost_pct": 0.001, 
        "state_space": len(train.tic.unique()), 
        "stock_dim": len(train.tic.unique()), 
        "tech_indicator_list": config.INDICATORS, 
        "action_space": len(train.tic.unique()), 
        "reward_scaling": 1e-4,
    }
    env_train = portfolioAllocationEnv(df = train, is_test_set=False, **env_kwargs)
    env_trade = portfolioAllocationEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = minReturnConcernEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = minReturnConcernEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = mddConcernEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = mddConcernEnv(df = trade, is_test_set=True, **env_kwargs)

    # env_train = sharpeConcernEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = sharpeConcernEnv(df = trade, is_test_set=True, **env_kwargs)
    model_index = i
    
    # load model
    model_zip_path = f'{config.TRAINED_MODEL_PATH}/{exp}/{model_name}_{str(model_index)}.zip'
    if model_name == 'A2C':
        model = A2C.load(model_zip_path)
    elif model_name == 'PPO':
        model = PPO.load(model_zip_path)
    elif model_name == 'DDPG':
        model = DDPG.load(model_zip_path)
    elif model_name == 'TD3':
        model = TD3.load(model_zip_path)
    elif model_name == 'SAC':
        model = SAC.load(model_zip_path)

    # test
    df_daily_return, df_actions = test_portfolioAllocation(model, env_trade)

    # evaluate
    print("==============DRL Strategy Stats===========")
    DRL_returns, DRL_stats = getStats(df_daily_return)
    print(DRL_stats)

    print("==============Equal Weight Strategy Stats===========")
    equalWeight_actions = getEqualWeightActions(trade)
    df_equalWeight_return, df_portfolio_value = computeReturns(equalWeight_actions,trade)
    equalWeight_returns, equalWeight_stats = getStats(df_equalWeight_return)
    print(equalWeight_stats)

    print("==============Min-Variance Strategy Stats===========")
    minVariance, minVariance_stats = getMinVariance(trade)
    print(minVariance_stats)

    print("==============All In Stock Strategy Stats===========")
    all_stock_actions= getTicActions(trade, 'VTI')
    df_stock_return, df_stock_value = computeReturns(all_stock_actions,trade)
    stock_returns, stock_stats = getStats(df_stock_return)
    print(stock_stats)

    print("==============All In Debt Strategy Stats===========")
    all_debt_actions= getTicActions(trade, 'TLT')
    df_debt_return, df_debt_value = computeReturns(all_debt_actions,trade)
    debt_returns, debt_stats = getStats(df_debt_return)
    print(debt_stats)

    print("==============All In Reit Strategy Stats===========")
    all_reit_actions= getTicActions(trade, 'VNQ')
    df_reit_return, df_reit_value = computeReturns(all_reit_actions,trade)
    reit_returns, reit_stats = getStats(df_reit_return)
    print(reit_stats)
    
    df_stats = pd.concat({'DRL': DRL_stats,
                          'equalWeight': equalWeight_stats,
                          'minVariance':minVariance_stats,
                          'stock': stock_stats,
                          'debt': debt_stats,
                          'reit': reit_stats},axis=1)

    df_annualReturn_allYear = stats_allYear(i, 'Annual return', df_stats, df_annualReturn_allYear)
    df_mdd_allYear = stats_allYear(i, 'Max drawdown', df_stats, df_mdd_allYear)
    df_sharpe_allYear = stats_allYear(i, 'Sharpe ratio', df_stats, df_sharpe_allYear)
    df_sortino_allYear = stats_allYear(i, 'Sortino ratio', df_stats, df_sortino_allYear)           
    df_rank_allYear = rankCaculate(i, 'Sharpe ratio', df_stats, df_rank_allYear)
    df_rank_allYear_s = rankCaculate(i, 'Sortino ratio', df_stats, df_rank_allYear)
    


In [ ]:
average_allYear(df_annualReturn_allYear)

In [ ]:
average_allYear(df_mdd_allYear)

In [ ]:
average_allYear(df_sharpe_allYear)
# df_sharpe_allYear.T.mean(axis=1).to_frame()

In [ ]:
average_allYear(df_sortino_allYear)

In [ ]:
average_allYear(df_rank_allYear)
# save_path = os.path.join(config.RESULTS_DIR, 'rank')
# df_rank.to_csv(f'{save_path}df_sharp_meanRank_{exp}.csv')

In [ ]:
average_allYear(df_rank_allYear_s)